In [16]:
import cv2
import numpy as np
import mediapipe as mp

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

def detect_batter_shot(landmarks):
    if landmarks:
        right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
        left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
        right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
        left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    
        if right_wrist.y < right_elbow.y and left_wrist.y < left_elbow.y:
            if right_wrist.x < left_shoulder.x:
                return "Cover Drive"
            elif right_wrist.x > right_shoulder.x:
                return "On Drive"
            else:
                return "Straight Drive"
        elif right_wrist.x < left_wrist.x:
            if right_wrist.y > right_elbow.y:
                return "Sweep"
            else:
                return "Square Drive"
        elif right_wrist.y > right_elbow.y and left_wrist.y > left_elbow.y:
            return "Reverse Sweep"
        elif right_wrist.x > left_wrist.x:
            return "Cut Shot"
        else:
            return "Forward Defence"
    return "Unknown"

def select_roi(frame):
    r = cv2.selectROI("Select Batter", frame, fromCenter=False, showCrosshair=True)
    cv2.destroyWindow("Select Batter")
    return r

cap = cv2.VideoCapture("C:\\Users\\chand\\Downloads\\bat_strokes.mp4")
ret, frame = cap.read()
if not ret:
    print("Failed to read video.")
    cap.release()
    exit()

x, y, w, h = select_roi(frame)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    roi = frame[y:y+h, x:x+w]
    frame_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    results = pose.process(frame_rgb)
    
    shot_type = "Unknown"
    if results.pose_landmarks:
        shot_type = detect_batter_shot(results.pose_landmarks.landmark)
    
    cv2.putText(frame, shot_type, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    cv2.imshow("Batter Shot Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
